### Fera formidável 4.5 - Um momento, por favor!

#### Enunciado

Objetivo: implemente o otimizador de Descida do Gradiente com Momento (Gradient
Descent with Momentum) na rede neural feita em Python puro.

Comentário: observe que o enunciado diz claramente que é para realizar a tarefa na
rede neural feita em Python puro nos vídeos da disciplina. Se você está usando o PyTorch,
numpy, tensorflow, keras, lightning ou qualquer outra biblioteca pronta, você está no
caminho errado!

In [210]:
import math 
import numpy as np
import random
import time
import seaborn as sns

In [211]:
random.seed(10)

In [212]:
class Valor:
    def __init__(self, data, progenitor=(), operador_mae="", rotulo=""):
        self.data = data
        self.progenitor = progenitor
        self.operador_mae = operador_mae
        self.rotulo = rotulo
        self.grad = 0

    def __repr__(self):
        return f"Valor(data={self.data})"
    
    def __add__(self, outro_valor):
        """Realiza a operação: self + outro_valor."""
        
        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data + outro_valor.data
        operador_mae = "+"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_adicao():
            self.grad += resultado.grad
            outro_valor.grad += resultado.grad
            
        resultado.propagar = propagar_adicao
        
        return resultado
    
    def __mul__(self, outro_valor):
        """Realiza a operação: self * outro_valor."""
        
        if not isinstance(outro_valor, Valor):
            outro_valor = Valor(outro_valor)
            
        progenitor = (self, outro_valor)
        data = self.data * outro_valor.data
        operador_mae = "*"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_multiplicacao():
            self.grad += resultado.grad * outro_valor.data # grad_filho * derivada filho em relação a mãe
            outro_valor.grad += resultado.grad * self.data
            
        resultado.propagar = propagar_multiplicacao
        
        return resultado
    
    def exp(self):
        """Realiza a operação: exp(self)"""
        progenitor = (self, )
        data = math.exp(self.data)
        operador_mae = "exp"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_exp():
            self.grad += resultado.grad * data 
        
        resultado.propagar = propagar_exp
        
        return resultado
    
    def __pow__(self, expoente):
        """Realiza a operação: self ** expoente"""
        assert isinstance(expoente, (int, float))
        progenitor = (self, )
        data = self.data ** expoente
        operador_mae = f"**{expoente}"
        resultado = Valor(data, progenitor, operador_mae)
        
        def propagar_pow():
            self.grad += resultado.grad * (expoente * self.data ** (expoente - 1))
        
        resultado.propagar = propagar_pow
        
        return resultado
    
    def __truediv__(self, outro_valor):
        """Realiza a operação: self / outro_valor"""
        return self * outro_valor ** (-1)
    
    def __neg__(self):
        """Realiza a operação: -self"""
        return self * -1
    
    def __sub__(self, outro_valor):
        """Realiza a operação: self - outro_valor"""
        return self + (-outro_valor)
    
    def __radd__(self, outro_valor):
        """Realiza a operação: outro_valor + self"""
        return self + outro_valor
    
    def __rmul__(self, outro_valor):
        """Realiza a operação: outro_valor * self"""
        return self * outro_valor
    
    def sig(self):
        """Realiza a operação: exp(self) / (exp(self) + 1)"""
        return self.exp() / (self.exp() + 1)
    
    def propagar(self):
        pass
    
    def propagar_tudo(self):
        
        self.grad = 1
        
        ordem_topologica = []
        
        visitados = set()

        def constroi_ordem_topologica(v):
            if v not in visitados:
                visitados.add(v)
                for progenitor in v.progenitor:
                    constroi_ordem_topologica(progenitor)
                ordem_topologica.append(v)

        constroi_ordem_topologica(self)
        
        for vertice in reversed(ordem_topologica):
            vertice.propagar()

In [213]:
class Neuronio:
    def __init__(self, num_dados_entrada):
        self.vies = Valor(random.uniform(-1, 1))
        
        self.pesos = []
        for i in range(num_dados_entrada):
            self.pesos.append(Valor(random.uniform(-1, 1)))
            
    def __call__(self, x):
        
        assert len(x) == len(self.pesos)
        
        soma = 0
        for info_entrada, peso_interno in zip(x, self.pesos):
            soma += info_entrada * peso_interno
            
        soma += self.vies  
        dado_de_saida = soma.sig()
        
        return dado_de_saida       
    
    def parametros(self):
        return self.pesos + [self.vies]

In [214]:
class Camada:
    def __init__(self, num_neuronios, num_dados_entrada):
        neuronios = []
        
        for _ in range(num_neuronios):
            neuronio = Neuronio(num_dados_entrada)
            neuronios.append(neuronio)
            
        self.neuronios = neuronios     
        
    def __call__(self, x):
        dados_de_saida = []
        
        for neuronio in self.neuronios:
            informacao = neuronio(x)
            dados_de_saida.append(informacao)
            
        if len(dados_de_saida) == 1:
            return dados_de_saida[0]
        else:        
            return dados_de_saida  
    
    def parametros(self):
        params = []
        
        for neuronio in self.neuronios:
            params_neuronio = neuronio.parametros()
            params.extend(params_neuronio)
        
        return params

In [215]:
class MLP:
    def __init__(self, num_dados_entrada, num_neuronios_por_camada):
        
        percurso = [num_dados_entrada] + num_neuronios_por_camada
        
        camadas = []
        
        for i in range(len(num_neuronios_por_camada)):
            camada = Camada(num_neuronios_por_camada[i], percurso[i])
            camadas.append(camada)
            
        self.camadas = camadas
        
    def __call__(self, x):
        for camada in self.camadas:
            x = camada(x)
        return x
    
    def parametros(self):
        params = []
        
        for camada in self.camadas:
            parametros_camada = camada.parametros()
            params.extend(parametros_camada)
            
        return params

In [216]:
NUM_DADOS_DE_ENTRADA = 3  
NUM_DADOS_DE_SAIDA = 1    
CAMADAS_OCULTAS = [3, 2]  

arquitetura_da_rede = CAMADAS_OCULTAS + [NUM_DADOS_DE_SAIDA]

minha_mlp = MLP(NUM_DADOS_DE_ENTRADA, arquitetura_da_rede)

In [217]:
df = sns.load_dataset("iris")
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [218]:

sepal_lenght = df["sepal_length"].values.tolist()
sepal_width = df["sepal_width"].values.tolist()
petal_lenght = df["petal_length"].values.tolist()
petal_width = df["petal_width"].values.tolist()

x = []
for sep_l, sep_w, pet_l in zip(sepal_lenght, sepal_width, petal_lenght):
  x.append([sep_l, sep_w, pet_l])
        
y_true = []
for i in petal_width:
  y_true.append(i)

print(x)
print(y_true)

[[5.1, 3.5, 1.4], [4.9, 3.0, 1.4], [4.7, 3.2, 1.3], [4.6, 3.1, 1.5], [5.0, 3.6, 1.4], [5.4, 3.9, 1.7], [4.6, 3.4, 1.4], [5.0, 3.4, 1.5], [4.4, 2.9, 1.4], [4.9, 3.1, 1.5], [5.4, 3.7, 1.5], [4.8, 3.4, 1.6], [4.8, 3.0, 1.4], [4.3, 3.0, 1.1], [5.8, 4.0, 1.2], [5.7, 4.4, 1.5], [5.4, 3.9, 1.3], [5.1, 3.5, 1.4], [5.7, 3.8, 1.7], [5.1, 3.8, 1.5], [5.4, 3.4, 1.7], [5.1, 3.7, 1.5], [4.6, 3.6, 1.0], [5.1, 3.3, 1.7], [4.8, 3.4, 1.9], [5.0, 3.0, 1.6], [5.0, 3.4, 1.6], [5.2, 3.5, 1.5], [5.2, 3.4, 1.4], [4.7, 3.2, 1.6], [4.8, 3.1, 1.6], [5.4, 3.4, 1.5], [5.2, 4.1, 1.5], [5.5, 4.2, 1.4], [4.9, 3.1, 1.5], [5.0, 3.2, 1.2], [5.5, 3.5, 1.3], [4.9, 3.6, 1.4], [4.4, 3.0, 1.3], [5.1, 3.4, 1.5], [5.0, 3.5, 1.3], [4.5, 2.3, 1.3], [4.4, 3.2, 1.3], [5.0, 3.5, 1.6], [5.1, 3.8, 1.9], [4.8, 3.0, 1.4], [5.1, 3.8, 1.6], [4.6, 3.2, 1.4], [5.3, 3.7, 1.5], [5.0, 3.3, 1.4], [7.0, 3.2, 4.7], [6.4, 3.2, 4.5], [6.9, 3.1, 4.9], [5.5, 2.3, 4.0], [6.5, 2.8, 4.6], [5.7, 2.8, 4.5], [6.3, 3.3, 4.7], [4.9, 2.4, 3.3], [6.6, 2.9, 4.

In [219]:
NUM_EPOCAS = 100
TAXA_DE_APRENDIZADO = 0.01
COEFICIENTE_DE_MOMENTO = 0.9

inicio = time.time()
velocidades = {}
for p in minha_mlp.parametros():
    velocidades[p] = 0
    
for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = []
    for exemplo in x:
        previsao = minha_mlp(exemplo)
        y_pred.append(previsao)

    # loss
    erros = []
    for yt, yp in zip(y_true, y_pred):
        residuo = yp - yt
        erro_quadratico = residuo ** 2
        erros.append(erro_quadratico)        
    loss = sum(erros)

    # zero grad
    for p in minha_mlp.parametros():
        p.grad = 0

    # backpropagation
    loss.propagar_tudo()

    # atualiza parâmetros
    for p in minha_mlp.parametros():
        vel = velocidades[p]
        vel = COEFICIENTE_DE_MOMENTO * vel - p.grad * TAXA_DE_APRENDIZADO
        p.data += vel
        velocidades[p] = vel

    # mostra resultado (opcional)
    loss = (sum(erros) / len(erros)) ** 0.5
    print(epoca, loss.data)
    
fim = time.time()
tempo = fim - inicio
print()
print(tempo)

0 0.986389190882295
1 0.8696018609950984
2 0.8066335182287651
3 0.7909979756616916
4 0.7871254206428763
5 0.7860119471393412
6 0.785646062371012
7 0.7855116438502096
8 0.7854573129333811
9 0.7854334458551911
10 0.7854221619398413
11 0.7854164669439895
12 0.7854134198482966
13 0.7854117018702693
14 0.7854106866099894
15 0.7854100607051256
16 0.7854096598681264
17 0.7854093942243177
18 0.7854092126673456
19 0.7854090850927653
20 0.7854089931869385
21 0.7854089254758939
22 0.785408874574217
23 0.7854088356089625
24 0.7854088052909147
25 0.7854087813528702
26 0.7854087622016074
27 0.7854087466973864
28 0.7854087340113114
29 0.7854087235312729
30 0.7854087147987937
31 0.7854087074659146
32 0.7854087012652932
33 0.7854086959891654
34 0.7854086914743365
35 0.7854086875913376
36 0.7854086842364943
37 0.7854086813260615
38 0.7854086787918394
39 0.785408676577863
40 0.7854086746378796
41 0.7854086729334067
42 0.7854086714322271
43 0.7854086701072133
44 0.7854086689353974
45 0.7854086678972364
46